In [7]:
from __future__ import print_function, division
from dataclasses import dataclass, replace
from pathlib import Path
from typing import Callable, List, Optional, Sequence, Union

import pandas as pd
import torch 
import random
import os
from pandas import DataFrame
from PIL import Image
from torch.utils.data.dataloader import default_collate
import sklearn
import shutil
from sklearn.metrics import f1_score, roc_auc_score, precision_score

import numpy as np
from matplotlib import pyplot as plt

import numpy as np
from torchvision import transforms

from torchvision import models
from torch.nn.modules import loss
from torch import optim
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torchvision.transforms.functional as TF
from tqdm.auto import tqdm

import os
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, utils
import torch.nn.functional as tf
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode
from torchvision.transforms import RandomVerticalFlip, RandomCrop

from albumentations import RandomCrop, Normalize, HorizontalFlip, Resize
from albumentations import Compose
from albumentations.pytorch import ToTensor

import datetime
import sys
sys.path.append('../src/')
from utils import *
from prepare_data import *
from model import *
import shutil
import argparse



In [15]:
data = pd.read_csv('../data/data_uf.csv')

In [13]:
50-18


32

In [12]:
data


,Folder,Id,Field,Well,CoringTop,CoringDown,CoringTopBind,CoringDownBind,CoreRecovery,PhotoTop,PhotoDown,PhotoType,LayerTop,LayerDown,Rock,Carbonate,Ruin,Saturation
0,Unload1,1000000,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,0.00,1.00,ДС,0.00,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные
1,Unload1,1000001,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,0.00,1.00,УФ,0.00,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные
2,Unload1,1000002,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,1.00,1.45,ДС,0.00,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные
3,Unload1,1000003,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,1.00,1.45,УФ,0.00,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные
4,Unload1,1000004,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,1.45,2.00,ДС,1.45,3.54,алевролит,не карбонатный,не разрушен,не опред.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6409,Unload2,1005001,Field2,Well10,2977.0,2989.5,2960.9,2973.4,12.50,11.00,11.18,УФ,8.00,11.18,"пересл. песчаника, алевролита и глин",не карбонатный,частично разрушен,не опред.
6410,Unload2,1005002,Field2,Well10,2977.0,2989.5,2960.9,2973.4,12.50,11.18,12.00,ДС,11.18,12.00,аргиллит,не карбонатный,частично разрушен,не опред.
6411,Unload2,1005003,Field2,Well10,2977.0,2989.5,2960.9,2973.4,12.50,11.18,12.00,УФ,11.18,12.00,аргиллит,не карбонатный,частично разрушен,не опред.
6412,Unload2,1005004,Field2,Well10,2977.0,2989.5,2960.9,2973.4,12.50,12.00,12.50,ДС,12.00,12.50,алевролит,не карбонатный,частично разрушен,не опред.


In [14]:
pd.get_dummies(data, columns=['Rock', 'Saturation'], prefix = ['rock', 'satur'])

,Folder,Id,Field,Well,CoringTop,CoringDown,CoringTopBind,CoringDownBind,CoreRecovery,PhotoTop,...,rock_песчаник,rock_песчаник глинистый,rock_уголь,rock_фундамент,satur_битуминозный,satur_не опред.,satur_нефтенасыщенные,satur_продукт,satur_пятнисто нефтенасыщенные,satur_слабо нефтенасыщенные
0,Unload1,1000000,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,0.00,...,1,0,0,0,0,0,1,0,0,0
1,Unload1,1000001,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,0.00,...,1,0,0,0,0,0,1,0,0,0
2,Unload1,1000002,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,1.00,...,1,0,0,0,0,0,1,0,0,0
3,Unload1,1000003,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,1.00,...,1,0,0,0,0,0,1,0,0,0
4,Unload1,1000004,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,1.45,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6409,Unload2,1005001,Field2,Well10,2977.0,2989.5,2960.9,2973.4,12.50,11.00,...,0,0,0,0,0,1,0,0,0,0
6410,Unload2,1005002,Field2,Well10,2977.0,2989.5,2960.9,2973.4,12.50,11.18,...,0,0,0,0,0,1,0,0,0,0
6411,Unload2,1005003,Field2,Well10,2977.0,2989.5,2960.9,2973.4,12.50,11.18,...,0,0,0,0,0,1,0,0,0,0
6412,Unload2,1005004,Field2,Well10,2977.0,2989.5,2960.9,2973.4,12.50,12.00,...,0,0,0,0,0,1,0,0,0,0


In [9]:
data.head()

,Folder,Id,Field,Well,CoringTop,CoringDown,CoringTopBind,CoringDownBind,CoreRecovery,PhotoTop,PhotoDown,PhotoType,LayerTop,LayerDown,Rock,Carbonate,Ruin,Saturation
0,Unload1,1000000,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,0.00,1.00,ДС,0.00,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные
1,Unload1,1000001,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,0.00,1.00,УФ,0.00,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные
2,Unload1,1000002,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,1.00,1.45,ДС,0.00,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные
3,Unload1,1000003,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,1.00,1.45,УФ,0.00,1.45,песчаник,не карбонатный,не разрушен,нефтенасыщенные
4,Unload1,1000004,Field6,Well11,1957.1,1963.1,1958.3,1964.3,5.93,1.45,2.00,ДС,1.45,3.54,алевролит,не карбонатный,не разрушен,не опред.


In [16]:
data['class'].value_counts()

0    2298
1     334
4     179
2     175
3     163
5      58
Name: class, dtype: int64

In [19]:
labels_list = []
for i in data['class'].value_counts():
    labels_list.append(1/i)

In [20]:
labels_list

[0.0004351610095735422,
 0.0029940119760479044,
 0.00558659217877095,
 0.005714285714285714,
 0.006134969325153374,
 0.017241379310344827]